In [1]:
import numpy as np
from pc_kriging import PC_Kriging
import matplotlib.pyplot as plt
from scipy.stats import norm
from doepy import build
from scipy import optimize
import pandas as pd
from numpy import genfromtxt
import pickle

from scipy.special import eval_legendre, eval_hermitenorm
import scipy.special

# adaptive learning - expected feasiability function --------------------------------------------------------------

def EFF(u,v,z):
    zl=-2*v
    zh=2*v
    return ((u-z)*( 2*norm.cdf((z-u)/v) - norm.cdf((zl-u)/v) - norm.cdf((zh-u)/v)) 
           -(v)*( 2*norm.pdf((z-u)/v) - norm.pdf((zl-u)/v) - norm.pdf((zh-u)/v))  
           +(2*v)*(norm.cdf((zh-u)/v) - norm.cdf((zl-u)/v)))

def U_function(u, v):
    return np.abs(u)/v

def LinearNorm(x,oldmin,oldmax,newmin,newmax):    # scaling linearly X to new domain limits
    return newmin + ((x-oldmin)*(newmax-newmin)/(oldmax-oldmin))


def VoronoiCell(x,xn):   #given x [single value] return the index of the closest xn [array]
    dist=PCK1.distance(x.reshape(1,-1),xn)
    return np.argmin(dist)

# Limit State 1

In [2]:
# #ground truth function 1 ----------------------------------------------------
# https://rprepo.readthedocs.io/en/latest/reliability_problems.html#rp201

def gfun_53(x):
    """Performance function for reliability problem 53.

    Parameters
    ----------
        x : numpy.array of float(s)
            Values of independent variables: columns are the different parameters/random variables (x1, x2,...xn) and rows are different parameter/random variables sets for different calls.

    Returns
    -------
        g_val_sys : numpy.array of float(s)
            Performance function value for the system.
        g_val_comp : numpy.array of float(s)
            Performance function value for each component.
        msg : str
            Accompanying diagnostic message, e.g. warning.
    """
#     import numpy as np
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(5*x[:, 0]/2) + 2 - (x[:, 0]**2 + 4)*(x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

# Limit State 2

In [3]:
# ground truth function 2 ----------------------------------------------------
# https://rprepo.readthedocs.io/en/latest/reliability_problems.html#rp201

def LState2(x):
    #"""Modified Performance function for reliability problem 53""""
  
    # expected number of random variables/columns
    nrv_e = 2

    g = float('nan')
    msg = 'Ok'
    x = np.array(x, dtype='f')

    n_dim = len(x.shape)
    if n_dim == 1:
        x = np.array(x)[np.newaxis]
    elif n_dim > 2:
        msg = 'Only available for 1D and 2D arrays.'
        return float('nan'), float('nan'), msg

    nrv_p = x.shape[1]
    if nrv_p != nrv_e:
        msg = f'The number of random variables (x, columns) is expected to be {nrv_e} but {nrv_p} is provided!'
    else:
        g = np.sin(2.0*x[:, 0]) -0.5 - (x[:, 0]**2 + 4)*(-1.0*x[:, 1] - 1)/20

    g_val_sys = g
    g_val_comp = g
    return g_val_sys, g_val_comp, msg

In [4]:
config = {"pol_type": ['hermite', 'hermite']}   #design variables following normal distribution
PCK1 = PC_Kriging(config)
PCK_loo = PC_Kriging(config)    # for LOOCV with same 'config' as specified in the original model

## Initial training 

In [6]:
function_1 = gfun_53   #ground truth function 1 
function_2 = LState2   #ground truth function 2

dim = 2       # dimensionality

x1mean, x1sigma = 1.5 , 1.0  # normal distribution 
x2mean, x2sigma = 2.5 , 1.0  # normal distribution 

ntest = 5000  # test points

# TEST POINTS -------------------------------------------------
XR = np.zeros((int(ntest), dim))   #normalized test points
XN = np.zeros((int(ntest), dim))  #scaled test points
YN_1 = np.zeros(int(ntest))
YN_2 = np.zeros(int(ntest))
#variable 1 ---------------------------------------------------
XN[:,0] = np.random.normal(0,1,ntest)  
XR[:,0] = PCK1.scalehermite(XN[:,0], x1mean, x1sigma)  
#variable 2 ---------------------------------------------------
XN[:,1] = np.random.normal(0,1,ntest)  
XR[:,1] = PCK1.scalehermite(XN[:,1], x2mean, x2sigma)  

YN_1 = function_1(XR)[0]
YN_2 = function_2(XR)[0]

In [7]:
# objective function to optimize length scale --------------------------------------------------------
def L_Object (l):
    v = 5/2
    N = len(xn)
    R = PCK1.matern(xn , xn, l, v)
    detR = np.linalg.det(R)
    
    modelpar2 = PCK1.train(xn, yn, p, np.array([l,v]))    # returns B, sig2, InfoMatrix(phi) , PolyIndices(alpha)
    ### ------------------Theta_ by UQLab User Manual PCK(C. Lataniotis, D. Wicaksono, S. Marelli, B. Sudret)------------------------------
    sig2 = modelpar2[1].reshape(-1)
    # return 0.5*(np.log(detR)+ N*np.log(2*np.pi*sig2)+ N)

    ### ------------------Theta_ by MLE PCK(Schobi,Sudret,Wiart)------------------------------
    FB = PCK1.InfoMat @ modelpar2[0]
    ins = (yn-FB).reshape(-1)
    R_1 = np.linalg.inv(R)
    return ((ins.T) @ R_1 @ ins) * (1/N) * (detR**(1/N))

In [ ]:
fail_samples_1 = np.sum(YN_1 < 0 )
Pf_ref_1 = fail_samples_1/ntest

fail_samples_2 = np.sum(YN_2 < 0 )
Pf_ref_2 = fail_samples_2/ntest

Pf_ref_1, Pf_ref_2

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(XR[:,0],XR[:,1], YN_1)
ax.scatter(XR[:,0],XR[:,1], YN_2)             #plotting the last trained surrogate
# ax.scatter(xr[:,0],xr[:,1], yn, color='red')   #observation points 
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('G_System')

# Combined Active training - U - 2 LS

In [8]:
number_experiments = 15    
number_active_points = 40

results_file = '_CATBDiff_U_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
                   
        if ( points == 0 ):   
            meanMC = meanMC_1
            varMC = varMC_1               
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_1 == 0 ): 
            meanMC = meanMC_1
            varMC = varMC_1              
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_2 == 0 ): 
            meanMC = meanMC_2
            varMC = varMC_2                
            print ('LS 2')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        else:
            Pf1_new = fail_prob_SUMO_1
            Pf2_new = fail_prob_SUMO_2

            B1_old = - norm.ppf ( Pf1_old )
            B2_old = - norm.ppf ( Pf2_old )

            B1_new = - norm.ppf ( Pf1_new )
            B2_new = - norm.ppf ( Pf2_new )

            B_error1 = abs(B1_old - B1_new ) / B1_old
            B_error2 = abs(B2_old - B2_new ) / B2_old

            if (B_error1 > B_error2):

                meanMC = meanMC_1
                varMC = varMC_1               
                print (B_error1, B_error2, 'LS 1')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2

            else:

                meanMC = meanMC_2
                varMC = varMC_2               
                print (B_error1, B_error2, 'LS 2')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2
                

        ### Evaluating new points
        U_f = U_function(meanMC.reshape(-1), varMC.reshape(-1))
        
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


c:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


LS1:  Degree 1 e_LOO 3.340579728099612e-05 Pf_ref 0.03133 Pf 0.09373 CoV 0.00980
LS2:  Degree 1 e_LOO 3.40098320395822e-05 Pf_ref 0.14797 Pf 0.16023 CoV 0.00720
number of training points:  11 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0009989146416013739 Pf_ref 0.03087 Pf 0.00051 CoV 0.14000
LS2:  Degree 3 e_LOO 1.6682120877382963e-05 Pf_ref 0.14772 Pf 0.24417 CoV 0.00560
number of training points:  12 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0029915507935448843 Pf_ref 0.03205 Pf 0.00307 CoV 0.05700
LS2:  Degree 1 e_LOO 0.0012277343579716062 Pf_ref 0.14693 Pf 0.04515 CoV 0.01450
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0022569511932779298 Pf_ref 0.03164 Pf 0.00181 CoV 0.07430


c:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\scipy\optimize\optimize.py:283: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  "minimize step, clipping to bounds", RuntimeWarning)


LS2:  Degree 1 e_LOO 0.004263836094249737 Pf_ref 0.14831 Pf 0.05909 CoV 0.01260
number of training points:  14 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.12702650499360166 Pf_ref 0.03171 Pf 0.02553 CoV 0.01950
LS2:  Degree 1 e_LOO 0.025799101061218944 Pf_ref 0.15045 Pf 0.03163 CoV 0.01750
number of training points:  15 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.05016007045232731 Pf_ref 0.03198 Pf 0.02224 CoV 0.02100
LS2:  Degree 1 e_LOO 0.015813733296221304 Pf_ref 0.14798 Pf 0.02231 CoV 0.02090
number of training points:  16 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.03701430062764798 Pf_ref 0.03007 Pf 0.05514 CoV 0.01310
LS2:  Degree 2 e_LOO 0.021798308039560153 Pf_ref 0.14795 Pf 0.08492 CoV 0.01040
number of training points:  17 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.07615057314229721 Pf_ref 0.03203 Pf 0.05348 CoV 0.01330
LS2:  Degree 2 e_LOO 0.00022394368125544

c:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:222: RuntimeWarning: divide by zero encountered in double_scalars


LS2:  Degree 2 e_LOO 0.000835533445912599 Pf_ref 0.14867 Pf 0.0 CoV inf
number of training points:  12 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0006089686133179167 Pf_ref 0.03123 Pf 0.00349 CoV 0.05340
LS2:  Degree 2 e_LOO 0.00033794443230606867 Pf_ref 0.15095 Pf 0.0 CoV inf
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0013220726327578046 Pf_ref 0.0315 Pf 0.00411 CoV 0.04920
LS2:  Degree 1 e_LOO 0.00029506368521980226 Pf_ref 0.1469 Pf 0.01534 CoV 0.02530
number of training points:  14 -------------------------------------------------
LS1:  Degree 1 e_LOO 3.420602017517905e-05 Pf_ref 0.03166 Pf 0.00237 CoV 0.06490
LS2:  Degree 2 e_LOO 0.007881049250143324 Pf_ref 0.15146 Pf 0.00056 CoV 0.13360
number of training points:  15 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0016085816844668523 Pf_ref 0.03102 Pf 0.00101 CoV 0.09950
LS2:  Degree 2 e_LOO 0.0011580332874248037 Pf_

# Combined training - EFF - 2 LS

In [8]:
number_experiments = 15
number_active_points = 40

results_file = '_CATBDiff_EFF_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors

            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
                
        if ( points == 0 ):   
            meanMC = meanMC_1
            varMC = varMC_1               
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_1 == 0 ): 
            meanMC = meanMC_1
            varMC = varMC_1              
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_2 == 0 ): 
            meanMC = meanMC_2
            varMC = varMC_2                
            print ('LS 2')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        else:
            Pf1_new = fail_prob_SUMO_1
            Pf2_new = fail_prob_SUMO_2

            B1_old = - norm.ppf ( Pf1_old )
            B2_old = - norm.ppf ( Pf2_old )

            B1_new = - norm.ppf ( Pf1_new )
            B2_new = - norm.ppf ( Pf2_new )

            B_error1 = abs(B1_old - B1_new ) / B1_old
            B_error2 = abs(B2_old - B2_new ) / B2_old

            if (B_error1 > B_error2):

                meanMC = meanMC_1
                varMC = varMC_1               
                print (B_error1, B_error2, 'LS 1')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2

            else:

                meanMC = meanMC_2
                varMC = varMC_2               
                print (B_error1, B_error2, 'LS 2')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2
        
        
        ### Evaluating new points
        eff = EFF(meanMC.reshape(-1), varMC.reshape(-1) , 0 )  #(mean, variance, target) 
        
        xr = np.append(xr, MCinputs[np.argmax(eff)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

Experiment:  1 #################################################################


C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


LS1:  Degree 1 e_LOO 3.836931236001603e-06 Pf_ref 0.032 Pf 0.06413 CoV 0.01210
LS2:  Degree 1 e_LOO 4.240468345252064e-05 Pf_ref 0.14862 Pf 0.16308 CoV 0.00720
number of training points:  11 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.046620807954644565 Pf_ref 0.03198 Pf 0.13326 CoV 0.00810
LS2:  Degree 2 e_LOO 0.03357610546351106 Pf_ref 0.14861 Pf 0.13187 CoV 0.00810
number of training points:  12 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.016214618248466598 Pf_ref 0.03123 Pf 0.13193 CoV 0.00810
LS2:  Degree 2 e_LOO 0.08277592172385313 Pf_ref 0.15097 Pf 0.11813 CoV 0.00860
number of training points:  13 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.0392053227429953 Pf_ref 0.0312 Pf 0.05091 CoV 0.01370
LS2:  Degree 1 e_LOO 0.002578679943607183 Pf_ref 0.14773 Pf 0.1142 CoV 0.00880
number of training points:  14 -------------------------------------------------
LS1:  Degree 1 e_LOO 0.016481263749379624 

LS1:  Degree 3 e_LOO 3.5795824472839923e-06 Pf_ref 0.03065 Pf 0.02235 CoV 0.02090
LS2:  Degree 3 e_LOO 0.00013177247121531427 Pf_ref 0.14828 Pf 0.06509 CoV 0.01200
number of training points:  45 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.00022042912202636353 Pf_ref 0.03205 Pf 0.02338 CoV 0.02040
LS2:  Degree 3 e_LOO 3.715089806399571e-05 Pf_ref 0.14768 Pf 0.06506 CoV 0.01200
number of training points:  46 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.0002650687644817312 Pf_ref 0.03255 Pf 0.02393 CoV 0.02020
LS2:  Degree 3 e_LOO 2.040493265214328e-05 Pf_ref 0.14884 Pf 0.07981 CoV 0.01070
number of training points:  47 -------------------------------------------------
LS1:  Degree 3 e_LOO 9.194838528816545e-05 Pf_ref 0.03117 Pf 0.0233 CoV 0.02050
LS2:  Degree 3 e_LOO 6.416148887256715e-06 Pf_ref 0.14739 Pf 0.07743 CoV 0.01090
number of training points:  48 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.000

C:\Users\Administrator\anaconda3\envs\surrogate\lib\site-packages\scipy\optimize\optimize.py:283: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  "minimize step, clipping to bounds", RuntimeWarning)


LS2:  Degree 3 e_LOO 1.9658281255334124e-05 Pf_ref 0.14919 Pf 0.00879 CoV 0.03360
number of training points:  27 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.0002197216941082794 Pf_ref 0.03043 Pf 0.05773 CoV 0.01280
LS2:  Degree 3 e_LOO 4.5071720238985615e-05 Pf_ref 0.14733 Pf 0.01127 CoV 0.02960
number of training points:  28 -------------------------------------------------
LS1:  Degree 2 e_LOO 0.0017871079513898244 Pf_ref 0.03103 Pf 0.07457 CoV 0.01110
LS2:  Degree 3 e_LOO 3.986480516514611e-05 Pf_ref 0.14956 Pf 0.01117 CoV 0.02980
number of training points:  29 -------------------------------------------------
LS1:  Degree 3 e_LOO 0.000884118933772588 Pf_ref 0.03168 Pf 0.05343 CoV 0.01330
LS2:  Degree 3 e_LOO 9.363860758973731e-06 Pf_ref 0.14839 Pf 0.01182 CoV 0.02890
number of training points:  30 -------------------------------------------------
LS1:  Degree 3 e_LOO 7.593746453878194e-06 Pf_ref 0.03065 Pf 0.02675 CoV 0.01910
LS2:  Degree 4 e_LOO 6.2350

# Combined training - U - LOO - 2 LS

In [ ]:
number_experiments = 13    
number_active_points = 40

results_file = '_CAT_ULOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        e_lcv = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xr, xr)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))
                
                e_lcv[i] = (yn[i] - mean_loo[i])**2

            sumat_1[:,p-1] = np.divide(e_lcv, var_loo)      #LOO CV as proposed by Le Gratiet

            e_loo = np.mean (yn - mean_loo)**2              
            #sumat_1[:,p-1] = np.divide(e_loo, var_loo)      #LOO CV squared errors with MSE
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000
        LOOCV_1 = np.zeros(MCS_samples)
        LOOCV_2 = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
        
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV_1[k]= varMC_1[k]*(1+sumat_1[voro, opt_1])
            LOOCV_2[k]= varMC_2[k]*(1+sumat_2[voro, opt_2])
            
        ### Evaluating new points
        U_f_1 = U_function(meanMC_1.reshape(-1), LOOCV_1.reshape(-1))
        U_f_2 = U_function(meanMC_2.reshape(-1), LOOCV_2.reshape(-1))
        
        U_f = (U_f_1 + U_f_2) / 2    #average U
        
        xr = np.append(xr, MCinputs[np.argmin(U_f)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmin(U_f)]).reshape(-1,2)
        
        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))

# Combined training - EFF - LOO - 2 LS

In [ ]:
number_experiments = 15    
number_active_points = 40

results_file = '_CATBDiff_EFFLOO_' + str(number_active_points)   # index for results files

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '#################################################################' )
    
    ActiveTrain_1 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }
    ActiveTrain_2 = {}   #results file { Surrogate , Pf, CoV_Pf , eLoo , mse }

    #INITIAL design of experiments (LHS) passive training ####################################

    n = 10       # number of initial sampling

    xn = np.zeros((int(n), dim))      #normalized training points
    xr = np.zeros((int(n), dim))      #scaled training points
    yn = np.zeros((int(n)))           #observations

    # Check the variables limits for space-filling distribution
    Xdoe = build.space_filling_lhs( {'x1':[-1, 1],      
                                     'x2':[-1, 1],} , 
                                      num_samples = n )
    #------------------------------------------------------------
    xn[:,0] = Xdoe['x1']
    xn[:,1] = Xdoe['x2']
    xr[:,0] = PCK1.scalehermite(xn[:,0], x1mean, x1sigma)
    xr[:,1] = PCK1.scalehermite(xn[:,1], x2mean, x2sigma)

    yn_1 = function_1(xr)[0]
    yn_2 = function_2(xr)[0]
    
    # kernel hyperparameters-------------------------------------
    v = 5/2        #Gaussian process
    #truncation term-------------------------------------
    p_max = 5  #for each variable → same truncation , degree of expansion
    
    for points in range(number_active_points):

        # Selecting the smallest e_loo model (length and order)
        
        mse_results = np.zeros(p_max-1)
        opt_length_it = np.zeros(p_max-1)
        eloo_results = np.zeros(p_max-1)

        mean_loo = np.zeros(len(xn))
        var_loo = np.zeros(len(xn))
        sumat_1 = np.zeros((len(xn),(p_max-1)))
        sumat_2 = np.zeros((len(xn),(p_max-1)))

        dist = PCK1.distance(xn, xn)
        lmax = np.max(dist)
        lmin = np.min(dist[dist!=0])
        bounds = [(lmin, lmax)]

        results = dict()
        
        # ##############################################################################
        # Active training of Limit State 1 #############################################
    
#         print('AT Limit state 1 ')

        ModelName_1 = 'PCK1_' + str(len(xn))
        
        ModelName_1 = PC_Kriging(config)

        # yn is changing for each LS (inside L_Object )
        yn = yn_1    #observations
        YN = YN_1    #test points

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):

            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_1[:,p-1] = np.divide(e_loo, var_loo)
            
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar1 = ModelName_1.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_1.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)    #selected based 'eloo' instead 'mse'
        opt_1 = opt 
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar1 = ModelName_1.train (xn, yn, int(opt+1), theta_opt) 

        ## Pool of samples for MCS -----------------------------------
        MCS_samples = 100000
        LOOCV = np.zeros(MCS_samples)

        MCinputs_norm = np.zeros((int(MCS_samples), dim))
        MCinputs = np.zeros((int(MCS_samples), dim))

        MCinputs_norm[:,0] = np.random.normal(0, 1, size=int(MCS_samples))
        MCinputs_norm[:,1] = np.random.normal(0, 1, size=int(MCS_samples))

        MCinputs[:,0] = PCK1.scalehermite(MCinputs_norm[:,0], x1mean, x1sigma)  
        MCinputs[:,1] = PCK1.scalehermite(MCinputs_norm[:,1], x2mean, x2sigma)  
    
        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_1 = function_1(MCinputs)[0]  
        fail_samples_1 = np.sum(ymc_1 < 0 )
        Pf_ref_1 = fail_samples_1 / MCS_samples
        
            # surrogate ls
        meanMC_1, varMC_1 = ModelName_1.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_1 = np.sum(np.asarray(meanMC_1) < 0 )
        fail_prob_SUMO_1 = fail_samples_SUMO_1 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_1 ) / (fail_prob_SUMO_1 * MCS_samples) )

        print('LS1: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results), 'Pf_ref', Pf_ref_1 ,'Pf', fail_prob_SUMO_1 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_1[str(len(xn))+'points'] = ModelName_1 , fail_prob_SUMO_1 , cov_pf , np.min(eloo_results), np.min(mse_results)
        
        # ##############################################################################
        # Active training of Limit State 2 #############################################
        
#         print('Training Limit state 2 ')
        
        ModelName_2 = 'PCK2_' + str(len(xn))
        
        ModelName_2 = PC_Kriging(config)
        
        # yn is changing for each LS 
        yn = yn_2
        YN = YN_2
        # Selecting the smallest e_loo model (length and order) 

        # OPTIMAL SURROGATE MODEL -----------------------------------
        for p in range(1, p_max):
            
            # results = dict()
            # bounds = [(lmin, lmax)]
            
            results['shgo'] = optimize.shgo(L_Object, bounds)
            opt_length = results['shgo']['x'][0]

            theta = np.array([opt_length, v])

            #Generating PCK models for each reduced design of experiments 

            for i in range (0, len(xn) ):

                yn_loo= np.delete(yn,[i])                            #y_n-i      leaving element i out the observations 
                xr_loo= np.delete(xr,[i*2,i*2+1]).reshape(-1,dim)    #x1r_n-i   leaving element i out the inputs (xr)
                xn_loo= np.delete(xn,[i*2,i*2+1]).reshape(-1,dim)    #x_n-i     leaving element i out the nomalized inputs (xn)

                #training LOO
                modelpar_loo = PCK_loo.train (xn_loo , yn_loo , p , theta )

                #predicting LOO over each removed sample
                mean_loo[i], var_loo[i] = PCK_loo.predict_fast(xn[i].reshape(1,-1))

            e_loo = np.mean (yn - mean_loo)**2              #LOO CV squared errors
            sumat_2[:,p-1] = np.divide(e_loo, var_loo)
            eloo_results[p-1] = e_loo

            #--------------------------------------- error over a set of test points
            modelpar2 = ModelName_2.train (xn, yn, p, theta) 
            mean0, var0 = ModelName_2.predict_fast(XN)    # test points predictions mean, variance

            mse = np.mean ((YN - mean0)**2)

            mse_results[p-1] = mse
            opt_length_it[p-1] = opt_length

    #         print('Degree', p, 'MSE', "%.2f" % round(mse, 2) , 'e_LOO', "%.5f" % e_loo)

        ## training optimal model ----------------------------

        opt = np.argmin(eloo_results)
        opt_2 = opt  
        theta_opt = np.array([opt_length_it[opt], v]) 

        modelpar2 = ModelName_2.train (xn, yn, int(opt+1), theta_opt) 

        # Pf estimation ----------------------------------------------
        
            # ground truth ls
        ymc_2 = function_2(MCinputs)[0]  
        fail_samples_2 = np.sum(ymc_2 < 0 )
        Pf_ref_2 = fail_samples_2 / MCS_samples
        
            # surrogate ls
        meanMC_2, varMC_2 = ModelName_2.predict_fast(MCinputs_norm)    # mean, variance
        fail_samples_SUMO_2 = np.sum(np.asarray(meanMC_2) < 0 )
        fail_prob_SUMO_2 = fail_samples_SUMO_2 / MCS_samples

        cov_pf = np.sqrt((1 - fail_prob_SUMO_2 ) / (fail_prob_SUMO_2 * MCS_samples) )        
        
        print('LS2: ','Degree', int(opt+1), 'e_LOO', np.min(eloo_results) , 'Pf_ref', Pf_ref_2 ,'Pf', fail_prob_SUMO_2 , 'CoV', "%.5f" % round(cov_pf, 4))

        #saving results ----------------------------

        ActiveTrain_2[str(len(xn))+'points'] = ModelName_2 , fail_prob_SUMO_2 , cov_pf , np.min(eloo_results), np.min(mse_results)

        # ##############################################################################
                
        if ( points == 0 ):   
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                 
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_1 == 0 ): 
            meanMC = meanMC_1
            varMC = varMC_1
            sumat = sumat_1
            opt = opt_1                 
            print ('LS 1')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        elif ( fail_prob_SUMO_2 == 0 ): 
            meanMC = meanMC_2
            varMC = varMC_2
            sumat = sumat_2
            opt = opt_2                 
            print ('LS 2')

            Pf1_old = fail_prob_SUMO_1
            Pf2_old = fail_prob_SUMO_2

        else:
            Pf1_new = fail_prob_SUMO_1
            Pf2_new = fail_prob_SUMO_2

            B1_old = - norm.ppf ( Pf1_old )
            B2_old = - norm.ppf ( Pf2_old )

            B1_new = - norm.ppf ( Pf1_new )
            B2_new = - norm.ppf ( Pf2_new )

            B_error1 = abs(B1_old - B1_new ) / B1_old
            B_error2 = abs(B2_old - B2_new ) / B2_old

            if (B_error1 > B_error2):

                meanMC = meanMC_1
                varMC = varMC_1
                sumat = sumat_1
                opt = opt_1                 
                print (B_error1, B_error2, 'LS 1')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2

            else:

                meanMC = meanMC_2
                varMC = varMC_2
                sumat = sumat_2
                opt = opt_2                 
                print (B_error1, B_error2, 'LS 2')

                Pf1_old = fail_prob_SUMO_1
                Pf2_old = fail_prob_SUMO_2
                
        # LOO CV errors ###################################################
        #variance enhancement based LOO CV erros around voronoi cells
        for k in range (0, MCS_samples):               
            voro = VoronoiCell(MCinputs[k], xr)
            LOOCV[k]= varMC[k]*(1+sumat[voro, opt])     

        ### Evaluating new points
        eff = EFF(meanMC.reshape(-1), LOOCV.reshape(-1) , 0 )  #(mean, variance, target) 
        
        xr = np.append(xr, MCinputs[np.argmax(eff)]).reshape(-1,2)
        xn = np.append(xn, MCinputs_norm[np.argmax(eff)]).reshape(-1,2)

        yn_1 = function_1(xr)[0]
        yn_2 = function_2(xr)[0]
    
    
        print('number of training points: ', len(xn),'-------------------------------------------------')
    filename1 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS1.sav'

    pickle.dump(ActiveTrain_1, open(filename1, 'wb'))
    
    filename2 = 'Batch_'+ str(experiments+1) + results_file + 'p_LS2.sav'
    pickle.dump(ActiveTrain_2, open(filename2, 'wb'))